# Widzenie maszynowe
## Laboratorium 5 - Estymacja pozy obiektu
*Autor: Paweł Mendroch* - [Github](https://github.com/FrozenTear7/computer-vision-lab/tree/master/lab5)

Rozszerzenia, które postanowiłem zaimplementować to modyfikacja danych w datasecie oraz częściowa migracja do tensorflow 2.

Nie przeprowadziłem pełnej konwersji kodu z tensorflow 1 do wersji 2 z keras, przerobiłem natomiast skrypt na wersję kompatybilną z nową biblioteką keras, bez potrzeby instalacji starszej wersji tensorflow i python, dzięki `tf.compat.v1`.

Dokonuję również augmentacji danych przy pomocy biblioteki `opencv`. Wykonuję 3 różne augmentacje:
- pojaśnienie obrazu
- przyciemnienie obrazu
- wyostrzenie obrazu

Nie wykonywałem innych prób, typu zmiana rozmiaru, obrót czy flipping, aby nie zepsuć danych związanych z położeniem.
Skrypt z poniższym kodem znajduje się w `prepareImages.py`, a całą obróbkę uruchamiam przy pomocy skryptu bashowego `augment_data.sh`, w którym przechowuję zmienną trybu augmentacji.

```
import os
import sys
import cv2
import numpy as np
import random


def augment_image(input_img, mode):
    print(mode)
    input_img = cv2.imread(input_img)

    if mode == 2:
        bright = np.ones(input_img.shape, dtype="uint8") * 70

        return cv2.subtract(input_img, bright)
    elif mode == 3:
        sharpening = np.array([[-1, -1, -1, [-1, 10, -1], [-1, -1, -1]]])

        return cv2.filter2D(image, -1, sharpening)
    else:
        bright = np.ones(input_img.shape, dtype="uint8") * 70

        return cv2.add(input_img, bright)


if __name__ == "__main__":
    in_dir = "./dataset/" + sys.argv[1]
    out_dir = "./dataset_new/" + sys.argv[1]
    augment_mode = sys.argv[2]

    for data_inner_dir in os.listdir(in_dir):
        try:
            for filename in os.listdir(os.path.join(in_dir, data_inner_dir)):
                if str(filename).endswith(".png"):
                    print(filename)
                    in_file = os.path.join(
                        os.path.join(in_dir, data_inner_dir), filename
                    )
                    out_file = os.path.join(
                        os.path.join(out_dir, data_inner_dir), filename
                    )

                    try:
                        os.remove(out_file)
                    except FileNotFoundError:
                        pass

                    image = augment_image(in_file, augment_mode)
                    cv2.imwrite(out_file, image)
        except NotADirectoryError:
            pass
```

Do testów uruchamiałem testy na 3 epokach, wynik dla oryginalnego skryptu wynosi: `accuracy - 0.4634`.

Data augmentation: 
- mode 1: `accuracy - 0.5219`.
- mode 2: `accuracy - 0.4780`.
- mode 3: `accuracy - 0.4731`.

Jak widać najlepiej wypadło rozjaśnienie obrazu, a wszystkie augmentacje dały lepsze wyniki niż oryginalne obrazy, lecz faktyczny test poniżej przeprowadzam dla 30 epok.

Original: `accuracy - 0.7756`

![Accuracy](./results/accuracy_history1.png)
![Loss](./results/loss_history1.png)
![Confusion](./results/Confusion_Matrix1.png)
![Pca](./results/pca1.png)

Data augmentation wybrałem wersję pierwszą, która dała dla mniejszej liczby epok najlepsze wyniki:

Brigther - mode 1: `accuracy - 0.7121`

Wyniki niestety mimo lepszych wyników początkowych dla małej ilości epok nie przełożyły się na dobre wyniki dla faktycznych testów. Zauważalny spadek około 6%.

![Accuracy](./results/accuracy_history1.png)
![Loss](./results/loss_history1.png)
![Confusion](./results/Confusion_Matrix1.png)
![Pca](./results/pca1.png)